In [1]:
from import_libraries.project_functions import *

warnings.filterwarnings("ignore")

# data from MT5

In [ ]:
symbol = 'EURUSD.pro'

In [ ]:
# def get_historical_data(timeframe=mt5.TIMEFRAME_D1, symbol="GER30", count=30_000):
#     """
#      Retrieves historical data from MetaTrader 5.

#      Arguments:
#      timeframe (int): Time frame for historical data.
#      symbol (str): The symbol for which you want to get historical data.
#      count (int): Number of candles requested.

#      Returns:
#      pd.DataFrame: DataFrame containing the received historical data.
#      """
#     if not connect_to_mt5():
#         print("Error connecting to MetaTrader 5.")
#         return None

#     try:
#         rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)
#     except Exception as e:
#         print(f"Error while receiving data: {e}")
#         mt5.shutdown()
#         return None

#     # Check the success of receiving data
#     if rates is not None:
#         # Convert data to pandas DataFrame
#         rates_frame = pd.DataFrame(rates)
#         rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
#         rates_frame = rates_frame.rename(columns=lambda x: x.capitalize())
#         rates_frame = rates_frame.rename(columns={"Time": "Date"})

#         rates_frame['Volume'] = rates_frame['High'] - rates_frame['Low']
#         rates_frame['MaxPositivePriceChange'] = rates_frame['High'] - rates_frame['Open']
#         rates_frame['MaxNegativePriceChange'] = rates_frame['Open'] - rates_frame['Low']

#         rates_frame['PriceChange'] = abs(rates_frame['Close'] - rates_frame['Close'].shift(1))

#         # Close the connection to MetaTrader 5
#         # mt5.shutdown()
#         return rates_frame
#     else:
#         print("Error while receiving data.")
#         mt5.shutdown()
#         return None




In [2]:
data = get_historical_data(symbol = symbol) # D1
# data_m5 = get_historical_data(timeframe=mt5.TIMEFRAME_M5, symbol="GER30", start='2022-01-01')
data.shape

Połączono z kontem


(14907, 12)

In [ ]:
# data.hist(figsize = (10,10));

# data preprocessing

In [ ]:
# preprocessor = Preprocessing_stock_data(data)
# all_indicator = preprocessor.all_methods()

# data = preprocessor.support_resistance_line()
# data

In [ ]:
# all_indicator.sample(3)

# experiments for lines


In [ ]:
# rebound_best_parameters = optimize_parameters(data, rebound_analysis, calculate_accumulated_price_changes,
#                                               windows_size= [i for i in range(1, 366, 1)],
#                                               bias= [i for i in range(1, 50, 1)]
#                                                 )

# print(f"Najlepsze opcje: {rebound_best_parameters}")
rebound_best_parameters = {'window_size': 2, 'bias': 10}

In [ ]:
# Create an instance of the class
genetic_algorithm = GeneticAlgorithm(data, rebound_analysis, calculate_accumulated_price_changes, 
                                     window_sizes= [i for i in range(1, 366, 1)], 
                                     biases= [i for i in range(1, 50, 1)])


# Run the genetic algorithm
best_individual = genetic_algorithm.run_genetic_algorithm(population_size=10, 
                                                          offspring_size=100, 
                                                          cx_probability=0.7, 
                                                          mut_probability=0.2, 
                                                          n_generations=50)

In [ ]:
current_data_optimize = define_level(data, rebound_best_parameters["window_size"] ,
                                  rebound_best_parameters["bias"])

rebound_data = rebound_analysis(current_data_optimize)
current_score = calculate_accumulated_price_changes(rebound_data, princ=True)
current_score

In [ ]:
sum(current_score) / (14907  /(22 * 12 )) / 12

In [ ]:
current_data_algorithm = define_level(data,
                ceil(genetic_algorithm.get_best_individual()[0]),
                ceil(genetic_algorithm.get_best_individual()[1]) )

rebound_data = rebound_analysis(current_data_algorithm)
current_score = calculate_accumulated_price_changes(rebound_data, princ=True)
current_score

In [ ]:
sum(current_score) / (14907  /(22 * 12 )) / 12

# Order

In [ ]:
automated_trading_from_signals(rebound_data, symbol= symbol)

# Experiments with Genetic Algorithm

In [ ]:
class GeneticAlgorithm:
    """
    A class representing a Genetic Algorithm for parameter optimization in trading strategies.

    Attributes:
        data (pd.DataFrame): The financial data used for analysis.
        analysis (callable): The analysis function to process the data.
        calculate (callable): The scoring function to evaluate the performance of parameter combinations.
        window_sizes (list): Possible values for the window size parameter.
        biases (list): Possible values for the bias parameter.
        best_individual (list): The best parameter combination found by the genetic algorithm.

    Methods:
        init_individual(individual_class): Initializes an individual for the genetic algorithm.
        evaluate(individual): Evaluates the fitness of an individual based on the provided scoring function.
        mutate(individual): Performs mutation on an individual to introduce diversity in the population.
        calculate_parameters(params): Calculates the score for a given set of parameters.
        run_genetic_algorithm(population_size, offspring_size, cx_probability, mut_probability, n_generations):
            Runs the genetic algorithm to find the best parameter combination.
        get_best_individual(): Returns the best parameter combination found by the genetic algorithm.
    """
    
    def __init__(self, data, analysis, calculate, window_sizes, biases=[1]):
        """Initializes the GeneticAlgorithm instance.

        Args:
            data (pd.DataFrame): The financial data used for analysis.
            analysis (callable): The analysis function to process the data.
            calculate (callable): The scoring function to evaluate the performance of parameter combinations.
            window_sizes (list): Possible values for the window size parameter.
            biases (list, optional): Possible values for the bias parameter. Defaults to [1].
        """
        # Initialize the GeneticAlgorithm instance with the provided parameters
        # Attribute Initialization:
        self.data = data
        self.analysis = analysis
        self.calculate = calculate
        self.window_sizes = window_sizes
        self.biases = biases
        self.best_individual = None

        # Problem Definition
        # Define the problem as a maximization problem
        creator.create("FitnessMax", base.Fitness, weights=(1.0,))
        creator.create("Individual", list, fitness=creator.FitnessMax)

        # Toolbox Initialization
        # Create a toolbox with the necessary components
        self.toolbox = base.Toolbox()

        # Registering Functions in the Toolbox
        # Register an initialization method for individuals
        self.toolbox.register("individual", self.init_individual, creator.Individual)
        
        # Register a method to initialize a population of individuals
        self.toolbox.register("population", tools.initRepeat, list, self.toolbox.individual)
        
        # Register the evaluation method for individuals
        self.toolbox.register("evaluate", self.evaluate)
        
        # Register the crossover method using Blend Crossover with a specified alpha value
        self.toolbox.register("mate", tools.cxBlend, alpha=0.5)
        
        # Register the mutation method
        self.toolbox.register("mutate", self.mutate)
        
        # Register the selection method using Tournament Selection with a tournament size of 3
        self.toolbox.register("select", tools.selTournament, tournsize=3)

    
    def init_individual(self, individual_class):
        """
        Initializes an individual for the genetic algorithm.

        Args:
            individual_class: The class representing an individual in the genetic algorithm.

        Returns:
            list: The initialized individual.
        """
        
        return individual_class([random.choice(self.window_sizes), random.choice(self.biases)])

    
    def evaluate(self, individual):
        """
        Evaluates the fitness of an individual based on the provided scoring function.

        Args:
            individual (list): The individual representing a parameter combination.

        Returns:
            tuple: The fitness score of the individual.
        """
        
        window_size, bias = map(int, individual)
        params = {'window_size': window_size, 'bias': bias}
        score = self.calculate_parameters(params)
        return (score,)

    
    def mutate(self, individual):
        """
        Performs mutation on an individual to introduce diversity in the population.

        Args:
            individual (list): The individual to be mutated.

        Returns:
            tuple: The mutated individual.
        """
        
        if random.random() < 0.5:
            individual[0] = int(abs(individual[0] + random.randint(-5, 5)))
        else:
            individual[1] = random.choice(self.biases)
        return individual,

    
    def calculate_parameters(self, params):
        """
        Calculates the score for a given set of parameters.

        Args:
            params (dict): The parameter values.

        Returns:
            float: The calculated score.
        """
        
        current_data = define_level(self.data, params['window_size'], params['bias'])
        rebound_data = self.analysis(current_data)
        return self.calculate(rebound_data)

        
    def run_genetic_algorithm(self, population_size=10, offspring_size=50, cx_probability=0.7, mut_probability=0.2, n_generations=10):
        """
        Runs the genetic algorithm to find the best parameter combination.

        Args:
            population_size (int, optional): The size of the initial population. Defaults to 10.
            offspring_size (int, optional): The size of the offspring population. Defaults to 50.
            cx_probability (float, optional): The crossover probability. Defaults to 0.7.
            mut_probability (float, optional): The mutation probability. Defaults to 0.2.
            n_generations (int, optional): The number of generations. Defaults to 10.

        Returns:
            list: The best parameter combination found by the genetic algorithm.
        """
        # Create an initial population
        population = self.toolbox.population(n=population_size)

        # Run the genetic algorithm
        algorithms.eaMuPlusLambda(population, self.toolbox, mu=population_size, lambda_=offspring_size,
                                  cxpb=cx_probability, mutpb=mut_probability, ngen=n_generations, stats=None, halloffame=None)

        # Get the best individual
        self.best_individual = tools.selBest(population, k=1)[0]
        print("Best Parameters:", self.best_individual)

        return self.best_individual

    
    def get_best_individual(self):
        """
        Returns the best parameter combination found by the genetic algorithm.

        Returns:
            list: The best parameter combination.
        """
        return self.best_individual


# Example usage of the GeneticAlgorithm class
# Create an instance of the class
genetic_algorithm = GeneticAlgorithm(data, rebound_analysis, calculate_accumulated_price_changes, 
                                     window_sizes= [i for i in range(1, 366, 1)], 
                                     biases= [i for i in range(1, 50, 1)])


# Run the genetic algorithm
best_individual = genetic_algorithm.run_genetic_algorithm(population_size=10, 
                                                          offspring_size=100, 
                                                          cx_probability=0.7, 
                                                          mut_probability=0.2, 
                                                          n_generations=50)

# AI

In [ ]:
def create_dataset(dataset, time_step=25):
    dataX, dataY = [], []
    dataset_without_close = dataset.drop('Close', axis=1)

    for i in range(len(dataset) - time_step - 1):
        if (i + time_step) < len(dataset):
            a = dataset_without_close.iloc[i:(i + time_step), :]
            dataX.append(a.values)
            dataY.append(dataset.iloc[i + time_step]['Close'])  # Добавлено изменение в индексации здесь
    return np.array(dataX), np.array(dataY)

In [ ]:
def create_dataset(dataset, time_step):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)